In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("data.csv")
#df

# Column Name: Make

In [3]:
df["make"]

0         \n,Audi ,\n
1         \n,Audi ,\n
2         \n,Audi ,\n
3         \n,Audi ,\n
4         \n,Audi ,\n
             ...     
62761    \n,Volvo ,\n
62762    \n,Volvo ,\n
62763    \n,Volvo ,\n
62764    \n,Volvo ,\n
62765    \n,Volvo ,\n
Name: make, Length: 62766, dtype: object

In [4]:
df["make"]=df["make"].str.strip(',\n')
df["make"]

0         Audi 
1         Audi 
2         Audi 
3         Audi 
4         Audi 
          ...  
62761    Volvo 
62762    Volvo 
62763    Volvo 
62764    Volvo 
62765    Volvo 
Name: make, Length: 62766, dtype: object

# Column Name: model

In [5]:
df["model"]

0         \n,A3,\n
1         \n,A3,\n
2         \n,A1,\n
3         \n,A1,\n
4         \n,A3,\n
           ...    
62761    \n,S60,\n
62762    \n,S60,\n
62763    \n,S60,\n
62764    \n,S40,\n
62765    \n,S60,\n
Name: model, Length: 62766, dtype: object

In [6]:
df["model"]=df["model"].str.strip(',\n')
df["model"]

0         A3
1         A3
2         A1
3         A1
4         A3
        ... 
62761    S60
62762    S60
62763    S60
62764    S40
62765    S60
Name: model, Length: 62766, dtype: object

# Column Name: seats

In [7]:
df["seats"]

0        \n5
1        \n5
2        \n5
3        \n4
4        \n5
        ... 
62761    \n5
62762    \n5
62763    \n5
62764    \n5
62765    \n5
Name: seats, Length: 62766, dtype: object

In [8]:
df["seats"]=df["seats"].str.strip(',\n')

In [9]:
df["seats"].replace("None", "0", inplace = True)

In [10]:
df["seats"]=df["seats"].astype(int)

df["seats"].head()

0    5
1    5
2    5
3    4
4    5
Name: seats, dtype: int32

# Column Name: doors

In [11]:
df["doors"].unique()

array(['\n3', '\n5', '\n4', 'None\n', '\n2', '\n6', '\n1'], dtype=object)

In [12]:
df["doors"]=df["doors"].str.strip(',\n')
df["doors"].replace("None", "0", inplace = True)
df["doors"]=df["doors"].astype(int)
df["doors"].head()

0    3
1    5
2    5
3    3
4    5
Name: doors, dtype: int32

# Column Name : seller

In [13]:
df['seller'] = df['seller'].str.lstrip("[', [' \s\S\t n ")
df['seller'] = df['seller'].str.rstrip("[', [] ' \s\S\t n")    
df["seller"]

0                Dealer
1                Dealer
2                Dealer
3                Dealer
4                Dealer
              ...      
62761            Dealer
62762            Dealer
62763            Dealer
62764    Private seller
62765            Dealer
Name: seller, Length: 62766, dtype: object

# Column Name : previous_owner

In [14]:
df['previous_owner'] = df['previous_owner'].str.lstrip("[' \s\S\t n ',]")
df['previous_owner'] = df['previous_owner'].str.rstrip("['\s,]")
df['previous_owner']

0        None
1        None
2        None
3        None
4        None
         ... 
62761    None
62762    None
62763       1
62764    None
62765    None
Name: previous_owner, Length: 62766, dtype: object

# Column Name : engine_size

In [15]:
# kolonun ici düzenlendi, none ifade = 0 a ayarlandi.
# ancak cc biriminden dolayi int cevrilmiyor, bu yüzden de mean() vs islemlr de yapilamiyor
#??????

In [16]:

df["engine_size"]=df["engine_size"].str.strip(',\n,')
df["engine_size"].replace("None", "0", inplace = True)
df["engine_size"].unique()

array(['1,984 cc', '1,395 cc', '1,495 cc', '999 cc', '1,390 cc',
       '1,798 cc', '1,197 cc', '3,197 cc', '2,773 cc', '2,000 cc',
       '1,399 cc', '1,398 cc', '3,189 cc', '2,995 cc', '3,199 cc',
       '1,595 cc', '2,480 cc', '1,498 cc', '1,781 cc', '1,998 cc',
       '1,500 cc', '3,993 cc', '3,184 cc', '1,598 cc', '1,800 cc', '0',
       '2,393 cc', '2,998 cc', '4,163 cc', '3,123 cc', '3,195 cc',
       '3,000 cc', '2,976 cc', '5,204 cc', '1,997 cc', '2,496 cc',
       '1,968 cc', '5,198 cc', '2,894 cc', '4,198 cc', '1,400 cc',
       '2,967 cc', '3,597 cc', '4,172 cc', '2,671 cc', '3,996 cc',
       '1,000 cc', '3,956 cc', '2,700 cc', '1,801 cc', '1,995 cc',
       '4,000 cc', '2,771 cc', '4,200 cc', '5,998 cc', '3,697 cc',
       '3,200 cc', '0 cc', '1,799 cc', '2,698 cc', '4,134 cc', '1,422 cc',
       '1,191 cc', '1,896 cc', '3,001 cc', '5,934 cc', '1 cc', '2,001 cc',
       '1,964 cc', '1,796 cc', '1,499 cc', '2,979 cc', '1,599 cc',
       '1,597 cc', '4,395 cc', '2,996 cc', 

# Column Name : gears

In [17]:
df["gears"].unique()

array(['\n,None,\n', '\n,7,\n', '\n,6,\n', '\n,8,\n', '\n,5,\n',
       '\n,1,\n', '\n,9,\n', '\n,4,\n', '\n,2,\n', '\n,3,\n'],
      dtype=object)

In [18]:
df["gears"]=df["gears"].str.strip(',\n,')
df["gears"].unique()

array(['None', '7', '6', '8', '5', '1', '9', '4', '2', '3'], dtype=object)

bi iki koda gerek var mi pek emin degilim,
Data missing de konusabiliriz!!!
df["gears"].replace("None", "0", inplace = True)
df["gears"]=df["gears"].astype(int)

# Column Name: cylinders

In [19]:
df["cylinders"].unique()

array(['\n,4,\n', '\n,3,\n', '\n,6,\n', '\n,None,\n', '\n,5,\n',
       '\n,8,\n', '\n,10,\n', '\n,12,\n', '\n,1,\n', '\n,2,\n', '\n,7,\n'],
      dtype=object)

In [20]:
df["cylinders"]=df["cylinders"].str.strip(',\n,')
df["cylinders"].unique()

array(['4', '3', '6', 'None', '5', '8', '10', '12', '1', '2', '7'],
      dtype=object)

# Column name : fuel_consumption

In [25]:
df['fuel_consumption'] = df['fuel_consumption'].str.lstrip(" [', \s\S\t\r\n ' ")
df['fuel_consumption'] = df['fuel_consumption'].str.rstrip("[', '] \s\S\t\r\n  ")
df['fuel_consumption'] = df['fuel_consumption'].str.replace("'","")
df['fuel_consumption']

0                                                     None
1                                     4.9 l/100 km (comb.)
2        0 l/100 km (comb.), 0 l/100 km (city), 0 l/100...
3        4.8 l/100 km (comb.), 5.9 l/100 km (city), 4.2...
4        5 l/100 km (comb.), 6.2 l/100 km (city), 4.3 l...
                               ...                        
62761                                                 None
62762                                                 None
62763    9.1 l/100 km (comb.), 12.6 l/100 km (city), 7 ...
62764                                                 None
62765    8.4 l/100 km (comb.), 11.6 l/100 km (city), 6....
Name: fuel_consumption, Length: 62766, dtype: object

# Column Name : production_date

In [22]:
df['production_date'] = pd.to_datetime(df['production_date'], format='%Y')
df['production_date']

0              NaT
1              NaT
2       2015-01-01
3              NaT
4              NaT
           ...    
62761          NaT
62762          NaT
62763          NaT
62764          NaT
62765          NaT
Name: production_date, Length: 62766, dtype: datetime64[ns]

# Column Name : Comfort & Convenience

In [23]:
df.rename({"\nComfort & Convenience\n":"comfort_convenience"}, inplace = True, axis = 1) # sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["comfort_convenience"] = df["comfort_convenience"].str.rstrip(" [', \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.replace("'","")
df["comfort_convenience"]

0        Air conditioning, Armrest, Automatic climate c...
1        Air conditioning, Automatic climate control, E...
2        Air conditioning, Armrest, Automatic climate c...
3        Air conditioning, Armrest, Automatic climate c...
4        Air conditioning, Armrest, Electrical side mir...
                               ...                        
62761    Air conditioning, Armrest, Automatic climate c...
62762    Air conditioning, Automatic climate control, C...
62763    Air conditioning, Armrest, Automatic climate c...
62764    Air conditioning, Armrest, Automatic climate c...
62765    Air conditioning, Armrest, Automatic climate c...
Name: comfort_convenience, Length: 62766, dtype: object